<a href="https://colab.research.google.com/github/NourEldin-Osama/T5_Fine-tuning_Text-simplification/blob/main/Notebooks/Text_simplification_Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install fastapi pyngrok uvicorn

In [ ]:
%%writefile utils.py
class TextSimplificationModel:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def simplify(self, text):
        text = "simplify: " + text
        encoded_input = self.tokenizer(text, return_tensors="pt").input_ids
        outputs = self.model.generate(encoded_input)
        outputs = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return outputs

Writing utils.py


In [ ]:
%%writefile main.py
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from typing import Union
from pydantic import BaseModel

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from utils import TextSimplificationModel

tokenizer = AutoTokenizer.from_pretrained("NourEldin-Osama/t5-small-finetuned-text-simplification")
model = AutoModelForSeq2SeqLM.from_pretrained("NourEldin-Osama/t5-small-finetuned-text-simplification")

app = FastAPI()
Model = TextSimplificationModel(tokenizer, model)

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class UserInput(BaseModel):
    text: str

@app.get('/')
def root():
    print("root() called")
    return {'message': 'API is Working 🚀'}


@app.post("/text_simplification/")
def text_simplification(request: UserInput):
    text = request.text
    output = Model.simplify(text)
    return {"output": output}

Writing main.py


In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
url = ngrok_tunnel.public_url
print('Public URL:', url)
print(f'Server Link: {url}' + r'/text_simplification/')

Public URL: https://6e7d-104-197-81-116.ngrok.io
Server Link: https://6e7d-104-197-81-116.ngrok.io/text_simplification/


In [ ]:
!uvicorn main:app --reload --host 127.0.0.1 --port 8000

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [528] using StatReload
INFO:     Started server process [531]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     105.32.77.69:0 - "GET /text_simplification/ HTTP/1.1" 405 Method Not Allowed
INFO:     105.32.77.69:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     197.47.176.25:0 - "GET /text_simplification/ HTTP/1.1" 405 Method Not Allowed
INFO:     197.47.176.25:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-05-04 1